In [1]:
import sys
import os
if not any(path.endswith('textbook') for path in sys.path):
    sys.path.append(os.path.abspath('../../..'))
from textbook_utils import *

(sec:linear_multi_fit)=
# Fitting the Multiple Linear Model

In the previous section we considered the case of two explanatory variables by using one variable called $x$ and the other $v$. Now we want to generalize the approach to $p$ explanatory variables. Our approach of choosing new letters to represent additional variables fails us. Instead, we use a more formal approach that represents multiple predictors as a matrix: 

$$
\begin{aligned}
\textbf{X} = 
\begin{bmatrix}
1 & x_{1,1} & x_{1,2} & \cdots & x_{1,p} \\
1 & x_{2,1} & x_{2,2} & \cdots & x_{2,p} \\
  &        & \vdots &        &        \\
1 & x_{n,1} & x_{n,2} & \cdots & x_{n,p} \\
\end{bmatrix}
\end{aligned}
$$

We call $\textbf{X}$ the *design matrix*. (Notice that $\textbf{X}$ is an $ n \times (p + 1)$ matrix.) Each column of $\textbf{X}$ represents a variable, and each row represents an observation. That is, $x_{i,j}$ is the measurement taken on observation $i$ of variable $j$.
For example, for a given observation, say, the second row in $\textbf{X}$, we represent the outcome $y_2$ by the linear approximation:

$$
\begin{aligned}
y_2 \approx  \theta_0 + \theta_1 x_{2,1} + \ldots + \theta_p x_{2,p} 
\end{aligned}
$$

Similar to the simple linear model, our
multiple linear model predicts $y_2$ for the given values $[x_{2,1}, \ldots,x_{2,p}]$.
But now, we have a collection of predictor (explanatory) variables.

We can write the model parameters as a $p+1$ column vector
${\boldsymbol{\theta}} = [ \theta_0, \theta_1, \ldots, \theta_p ]^t$.
We can also represent $\mathbf{y}$ as a column vector of the outcomes:

$$
\begin{aligned}
\mathbf{y} =  
\begin{bmatrix}
y_1 \\
y_2 \\
\vdots \\
y_n
\end{bmatrix}
\end{aligned}
$$

Putting these notational definitions together, we can write the vector of predictions for the entire dataset using matrix multiplication:

$$
\begin{aligned}
\hat{\mathbf{y}} &= {\textbf{X}} {\boldsymbol{\theta}}
\end{aligned}
$$

Checking the dimensions of $\textbf{X}$ and $\boldsymbol{\theta}$ confirms that $\hat{\mathbf{y}}$ is an $ n $-dimensional column vector. 
Each item in this vector is the model's prediction for one observation.
Additionally, the errors in using $\hat{\mathbf{y}}$ to predict $\mathbf{y}$ can be expressed as the $n$-dimensional column vector:

$$ \mathbf{e} = \mathbf{y}  - \hat{\mathbf{y}}.$$

Matrix notation will help us fit the multiple linear model.

Similar to the simple linear model, we fit ${\textbf{X}} {\boldsymbol{\theta}}$ using squared loss.
That is, we want to find the model parameters $\hat{\boldsymbol{\theta}}$ that minimize the average squared loss:

$$
\begin{aligned}
L({\boldsymbol{\theta}}, \textbf{X}, {\mathbf{y}})
 &= \frac{1}{n} \sum_i [y_i - (\theta_0 + \theta_1 x_{i,1} + \cdots + \theta_p x_{i,p})]^2 \\
 &= \frac{1}{n}  \lVert \mathbf{y} - {\textbf{X}} {\boldsymbol{\theta}} \rVert^2
\end{aligned}
$$

Here, we use the notation $ \lVert \mathbf{v} \rVert^2 $ for a vector $\mathbf{v}$ as a
shorthand for the sum of each vector element squared [^ell2]:
$\lVert \mathbf{v} \rVert^2 = \sum_i v_i^2$.

[^ell2]: $\lVert \mathbf{v} \rVert^2$ is also called the $\ell_2$ norm of a vector $\mathbf{v}$.

We can fit our model using calculus as we did for the simple linear model.
However, this approach can get cumbersome, and instead we use a geometric argument that is more intuitive and easily leads to useful properties of the design matrix, errors, and predicted values.

## A Geometric Problem

Again, our goal is the find the $ \hat{\theta} $ that minimizes our loss
function---we want to make $ L(\theta, X, y) $ as small as possible
for a given $ X $ and $ y $.
The key insight is that we can restate this goal in a geometric way.
Remember: the model predictions $ f_{\theta}(X) $ and the true outcomes
$ y $ are both vectors.
We can treat vectors as points---for example, we can plot
the vector $ [ 2, 3 ] $ at $ x = 2, y = 3 $ in 2D space.
Then, minimizing $ L(\theta, X, y) $ is equivalent to finding
$ \hat{\theta} $ that makes $ f_{\theta}(X) $ as close as possible to
$ y $ when we plot them as points.
As depicted in {numref}`Figure %s <fig:geom-2d>`, different values of
$ \theta $ give different predictions $ f_{\theta}(X) $ (hollow points).
Then, $ \hat{\theta} $ is the vector of parameters that put
$ f_{\theta}(X) $ as close to $ y $ (filled point) as possible.

```{figure} figures/geom-2d.svg
---
name: fig:geom-2d
width: 250px
---

A plot showing different values of $ f_{\theta}(X) $ (hollow points) and
the outcome vector $ y $ (filled point).
```

Next, we'll look at the possible values of $ f_{\theta}(X) $.
In {numref}`Figure %s <fig:geom-2d>`, we showed a few possible 
$ f_{\theta}(X) $.
Instead of just plotting a few possible points, we can
plot *all* possible values of $ f_{\theta}(X) $ by varying $ \theta $.
This results in a subspace of possible $ f_{\theta}(X) $ values, as shown in
{numref}`Figure %s <fig:geom-span>`.

```{figure} figures/geom-span.svg
---
name: fig:geom-span
width: 250px
---

A plot showing all possible values of $ f_{\theta}(X) $ as a line.
```

In the above {numref}`Figure %s <fig:geom-span>`, we drew the set of
possible $ f_{\theta}(X) $ values as a line.
Since our model is $ f_{\theta}(X) = X \theta $, from a property of
matrix-vector multiplication we know that $ f_{\theta}(X) $ is a linear
combination of the columns of $ X $, which we also call $ \text{span}(X) $.
Now, we need to figure out which point within $ \text{span}(X) $ lies the
closest to $ y $.

As {numref}`Figure %s <fig:geom-span>` suggests, the closest point to $ y $ 
is the point where the error $ \epsilon = y - f_{\theta}(X) $ is perpendicular
to $ \text{span}(X) $. We'll leave the complete proof as an exercise.
With this final fact, we can solve for $ \hat{\theta} $:

$$
\begin{aligned}
f_\hat{\theta}(X) + \epsilon &= y \\
X \hat{\theta} + \epsilon &= y \\
X^\top X \hat{\theta} + X^\top \epsilon &= X^\top y
    & (\text{left-multiply both sides by } X^\top) \\
X^\top X \hat{\theta} &= X^\top y
    & (X^\top \epsilon = 0 \text{ since } \epsilon \perp \text{span}(X)) \\
\hat{\theta} &= (X^\top X)^{-1} X^\top y
\end{aligned}
$$


With this derivation done, we can now write a short function to
fit the multiple linear model using $ X $ and $ y $.

In [3]:
def fit(X, y):
    return np.linalg.inv(X.T @ X) @ X.T @ y

Notice that deriving $ \hat{\theta} $ for the multiple linear model also gives
us $ \hat{\theta} $ for the simple linear model too. If we set
$ X $ to contain the intercept column and one column of features, using the
formula for $ \hat{\theta} $ gives us the intercept and slope of the best-fit
line.